# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from src.match_names import name_fbref_to_fpl, neutralize_name
from config import *

In [3]:
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]

In [4]:
len(FEATURES_OUTFIELD)

38

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (2,3,6,8,36,43,44,45,59,84,86,147,208) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df = df[~df[FEATURES_OUTFIELD].isnull().any(axis=1)] # for some reason 45 rows are missing some feature values

In [7]:
X_test = df[df["Season"] == CURRENT_SEASON][FEATURES_OUTFIELD]

In [8]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xG_30', 'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30',
       'Min_points_30', 'Team_result_30', 'DEF', 'FWD', 'MID', 'Finished'],
      dtype='object', length=273)

In [9]:
X_test.columns

Index(['Was_home', 'Rating_difference', 'Price', 'Transfers_balance',
       'Avg_FPL_points', 'Min_2', 'Gls_2', 'Ast_2', 'xG_2', 'xA_2',
       'Team_CS_2', 'Team_score_2', 'Opp_score_2', 'Team_result_2',
       'FPL_points_2', 'Min_4', 'Gls_4', 'Ast_4', 'xG_4', 'xA_4', 'Team_CS_4',
       'Team_score_4', 'Opp_score_4', 'Team_result_4', 'FPL_points_4',
       'Min_30', 'Gls_30', 'Ast_30', 'xG_30', 'xA_30', 'Team_CS_30',
       'Team_score_30', 'Opp_score_30', 'Team_result_30', 'FPL_points_30',
       'DEF', 'FWD', 'MID'],
      dtype='object')

In [10]:
# df[df["FPL_pos"] == "GK"]

In [11]:
def new_predict(X):
    XGB_COMPONENT = 0.45
    OVERALL_COMPONENT = 0.5
    FORM_COMPONENT = 0.25
    
    model_pred = np.array(model.predict2(X))
    overall_pred = np.array([row["Avg_FPL_points"] for i, row in X.iterrows()])
    form_pred = np.array([row["FPL_points_4"] for i, row in X.iterrows()])
    
    return np.add( model_pred * XGB_COMPONENT, overall_pred * OVERALL_COMPONENT, form_pred * FORM_COMPONENT )

file = open("models/GBR.pkl",'rb')
model = pickle.load(file)

In [12]:
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

# Predicting

In [13]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [14]:
def adjust_goalkeepers(df):
    gks = pd.read_csv("data/misc/goalkeepers.csv")
    gk_names = gks["Name"].to_list()
    gk_names = [name_fbref_to_fpl(neutralize_name(n)) for n in gk_names]
    # print(gk_names)
    df.loc[(df["FPL_pos"] == "GK") & ~(df["Name_original"].apply(neutralize_name).isin(gk_names)), "Pred"] = 0
    return df

In [15]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_goalkeepers(preds)
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [16]:
preds = get_predictions(model, df, X_test)

ValueError: feature_names mismatch: ['Was_home', 'Rating_difference', 'Value', 'Transfers_balance', 'Avg_FPL_points', 'Min_2', 'Gls_2', 'Ast_2', 'xG_2', 'xA_2', 'Team_CS_2', 'Team_score_2', 'Opp_score_2', 'Team_result_2', 'FPL_points_2', 'Min_4', 'Gls_4', 'Ast_4', 'xG_4', 'xA_4', 'Team_CS_4', 'Team_score_4', 'Opp_score_4', 'Team_result_4', 'FPL_points_4', 'Min_30', 'Gls_30', 'Ast_30', 'xG_30', 'xA_30', 'Team_CS_30', 'Team_score_30', 'Opp_score_30', 'Team_result_30', 'FPL_points_30', 'DEF', 'FWD', 'MID'] ['Was_home', 'Rating_difference', 'Price', 'Transfers_balance', 'Avg_FPL_points', 'Min_2', 'Gls_2', 'Ast_2', 'xG_2', 'xA_2', 'Team_CS_2', 'Team_score_2', 'Opp_score_2', 'Team_result_2', 'FPL_points_2', 'Min_4', 'Gls_4', 'Ast_4', 'xG_4', 'xA_4', 'Team_CS_4', 'Team_score_4', 'Opp_score_4', 'Team_result_4', 'FPL_points_4', 'Min_30', 'Gls_30', 'Ast_30', 'xG_30', 'xA_30', 'Team_CS_30', 'Team_score_30', 'Opp_score_30', 'Team_result_30', 'FPL_points_30', 'DEF', 'FWD', 'MID']
expected Value in 

In [ ]:
# preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == NEXT_GAMEWEEK)].head(30)

In [ ]:
# next gameweek
preds[preds["GW"] == NEXT_GAMEWEEK].head(20)

In [ ]:
# all preds
preds.head(20)

In [ ]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

# Saving to file

In [ ]:
preds.to_csv("predictions/preds_next_season.csv", index=False)

# Importance

In [ ]:
# import shap
# explainer = shap.Explainer(model.predict, X_test[9000:10000])
# shap_values = explainer(X_test[9000:10000])

In [ ]:
# shap.plots.bar(shap_values, max_display=15)

In [ ]:
# # Salah vs Nottingham Forest
# shap.plots.waterfall(shap_values[502])

In [ ]:
# shap.plots.waterfall(shap_values[470])